## **Tool Calling**

In [36]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, AnyMessage, ToolMessage
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [21]:
llm = ChatGroq(model="qwen/qwen3-32b")
llm.invoke(
    [HumanMessage(content="Test line.")]
).content

'<think>\nOkay, the user sent "Test line." I need to figure out how to respond. Since it\'s a test, maybe they\'re checking if the system is working. I should acknowledge their message and offer help.\n\nI\'ll start by confirming that I received their test line. Then, ask if they need assistance with anything specific. Keeping it friendly and open-ended makes sense. Let me make sure the response is clear and not too robotic. Yeah, that should work.\n</think>\n\nHello! I see you\'ve sent a test line. How can I assist you today? 😊'

In [22]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """this function is responsible for add tow number.
    
    Args:
        a: int
        b: int
    
    and return an integer number.
    """
    
    return a + b

In [23]:
tools = [add]

In [24]:
llm_with_tools = llm.bind_tools(tools)

In [27]:
response = llm_with_tools.invoke("Can you add this tow number 2, and 5")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '3czs0kmyy', 'function': {'arguments': '{"a":2,"b":5}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 178, 'prompt_tokens': 158, 'total_tokens': 336, 'completion_time': 0.270021485, 'prompt_time': 0.006019706, 'queue_time': 0.050783174, 'total_time': 0.276041191}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_8487b76fd3', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b2fe2a96-1c90-46a4-aede-350dcbadce84-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 5}, 'id': '3czs0kmyy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 158, 'output_tokens': 178, 'total_tokens': 336})

In [28]:
response.tool_calls

[{'name': 'add',
  'args': {'a': 2, 'b': 5},
  'id': '3czs0kmyy',
  'type': 'tool_call'}]

In [33]:
response.tool_calls[0]['args']

{'a': 2, 'b': 5}

In [34]:
add.invoke(response.tool_calls[0]['args'])

7

In [40]:
add.invoke(response.tool_calls[0])

ToolMessage(content='7', name='add', tool_call_id='3czs0kmyy')

In [43]:
llm_with_tools.invoke(
    [
        ToolMessage(
        tool_call_id = response.tool_calls[0]["id"],
        content=response.tool_calls[0]
        )
    ]
    
)

AIMessage(content='The sum of 2 and 5 is 7.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 189, 'total_tokens': 301, 'completion_time': 0.225658279, 'prompt_time': 0.007268358, 'queue_time': 0.088723071, 'total_time': 0.232926637}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_57dfb5ae7c', 'finish_reason': 'stop', 'logprobs': None}, id='run--3702384f-3563-4997-875d-3ac2ecf010e7-0', usage_metadata={'input_tokens': 189, 'output_tokens': 112, 'total_tokens': 301})